# Hyperparameter tuning in Keras Tensorflow

In [20]:
#!pip install -U tensorflow

import tensorflow as tf
from tensorflow import keras

tf.__version__


'2.19.0'

# Hyperparameter Tuning and Model Testing

In this notebook, we will explore the process of tuning machine learning models, specifically neural networks, by experimenting with various hyperparameters. We will examine how different settings such as the number of hidden layers, neurons per layer, learning rate, and optimization algorithms impact model performance.

The goal is to gain practical experience with both manual and automated hyperparameter optimization techniques, using tools like `RandomizedSearchCV` and `Keras Tuner`. Through this, we aim to build more accurate and stable models for predicting housing prices based on the California Housing dataset.

## 1. Data import

In [21]:
!pip install scikit-learn==1.5.2 scikeras --upgrade

Data is split into training ~60%, validation ~20%, and test ~20% sets without a fixed random seed, so splits may vary each run.

In [22]:
import sklearn
import numpy
import scipy

print("scikit-learn version:", sklearn.__version__)
print("numpy version:", numpy.__version__)
print("scipy version:", scipy.__version__)

scikit-learn version: 1.6.1
numpy version: 2.0.2
scipy version: 1.15.3


In [23]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target) # no random_state=42
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full) # no random_state=42
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_train.shape


ImportError: cannot import name '_IS_PYPY' from 'sklearn.utils.fixes' (/usr/local/lib/python3.11/dist-packages/sklearn/utils/fixes.py)

Here, we create a parameter grid for hyperparameter tuning. This defines the ranges and options for the number of hidden layers, neurons per layer, learning rate, and optimizer to be explored by the grid search.


In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
"model__n_hidden": [0, 1, 2, 3],
"model__n_neurons": np.arange(1, 100),
"model__learning_rate": reciprocal(3e-4, 3e-2),
"model__optimizer": ["adam", "sgd", "nesterov"]
}


This function builds a neural network model based on the given hyperparameters:

- `n_hidden`: number of hidden layers,
- `n_neurons`: number of neurons in each hidden layer,
- `optimizer`: choice of optimizer algorithm (`sgd`, `nesterov`, `momentum`, or `adam`),
- `learning_rate`: learning rate for the optimizer.

The model uses ReLU activation in hidden layers and a single output neuron for regression (predicting house prices). The optimizer is configured according to the selected type and learning rate. The model is compiled with mean squared error (MSE) loss.


In [ ]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=[8]))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
  model.add(keras.layers.Dense(1)) # because we estmate the price of a house so the output is one variable/value.

   # oplimalizator choice
  if optimizer == "sgd":
      opt = keras.optimizers.SGD(learning_rate=learning_rate)
  elif optimizer == "nesterov":
      opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
  elif optimizer == "momentum":
      opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
  elif optimizer == "adam":
      opt = keras.optimizers.Adam(learning_rate=learning_rate)
  else:
      raise ValueError(f"Nieznany optymalizator: {optimizer}")



  model.compile(loss="mse", optimizer=opt)
  return model

We create a TensorBoard callback to log training progress and metrics. This allows us to visualize the learning process in TensorBoard, making it easier to monitor model performance over time.


In [ ]:
import os
root_logdir = os.path.join(os.curdir, "house_logs")
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()


tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

Here we create and train a test model with 1 hidden layer of 100 neurons, using the Adam optimizer and a learning rate of 0.0003. Training runs for 20 epochs, and progress is logged with TensorBoard for visualization.

In [ ]:
model_clf = build_model(1,100,"adam", 3e-4)
history = model_clf.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks = [tensorboard_cb])

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=house_logs --port=6006

In this section, we train multiple models with different learning rates to observe their effect on training loss. We fix the number of hidden layers and neurons, vary the learning rate over several orders of magnitude, and plot the training loss curves on a logarithmic scale for comparison.


**Conclusions:**  
Increasing the learning rate generally speeds up training and leads to faster loss reduction. However, beyond a certain point, too high a learning rate causes instability in training, resulting in fluctuating or diverging loss values. Therefore, it is important to choose a learning rate that balances convergence speed and training stability.


In [ ]:
import matplotlib.pyplot as plt

learning_rates = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
h = []

for lr in learning_rates:
    model = build_model(n_hidden=1, n_neurons=100, optimizer="adam", learning_rate=lr)
    history = model.fit(X_train, y_train, epochs=40,
                        validation_data=(X_valid, y_valid),
                        verbose=0)
    h.append(history)

# visualization
for i, h_i in enumerate(h):
    plt.plot(h_i.history['loss'], label=f"Training loss 1e-{6 - i}")

plt.xlabel("Epoka")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.title("Porównanie strat dla różnych learning_rate")

plt.yscale('log')

plt.show()

In [ ]:
!pip install scikeras

In this part, we wrap our Keras model with `KerasRegressor` to make it compatible with scikit-learn tools. We set up an early stopping callback to prevent overfitting by stopping training if the validation loss does not improve for 10 epochs.

Then, we use `RandomizedSearchCV` to perform a randomized hyperparameter search over the predefined parameter grid. The search runs 5 iterations with 3-fold cross-validation, training each candidate model for up to 100 epochs with validation data.

This automated search helps us find the best combination of hyperparameters to improve model performance.

In [ ]:
import scikeras
from scikeras.wrappers import KerasRegressor

es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)

keras_reg = KerasRegressor( model=build_model,
    model__n_hidden=1,
    model__n_neurons=30,
    model__optimizer="adam",
    model__learning_rate=0.001,
    callbacks=[es])

In [ ]:
#!pip install scikit-learn==1.5.2 scikeras --upgrade

Here, we run the randomized hyperparameter search using `RandomizedSearchCV` with 5 iterations and 3-fold cross-validation. Each model is trained for up to 100 epochs with validation data used for early stopping and performance monitoring.

After the search completes, we save the best-found hyperparameters to a file (`rnd_search_params.pkl`) as well as the entire search object (`rnd_search_scikeras.pkl`) for later use or analysis.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs,n_iter=5, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

In [ ]:
import pickle

best_params = rnd_search_cv.best_params_
with open("rnd_search_params.pkl", "wb") as f:
    pickle.dump(best_params, f)

In [ ]:
with open("rnd_search_scikeras.pkl", "wb") as f:
    pickle.dump(rnd_search_cv, f)

Here we output the best combination of hyperparameters discovered by `RandomizedSearchCV`, which achieved the lowest validation loss during the search.


In [ ]:
rnd_search_cv.best_params_

In [ ]:
!pip install keras-tuner

# Description of the Hyperparameter Tuning Code

This code performs hyperparameter tuning for a neural network regression model using **Keras Tuner**.

- **Model building function (`build_model_kt`)**:  
  Defines a Keras Sequential model with a variable number of hidden layers (`n_hidden`) and neurons per layer (`n_neurons`).  
  The learning rate and optimizer type (`adam`, `sgd`, `nesterov`) are also tuned.

- **Keras Tuner Random Search**:  
  Initializes a tuner that tries up to 20 different hyperparameter configurations to minimize validation mean squared error (`val_mse`).  
  Uses callbacks:  
  - EarlyStopping (stops training early if no improvement)  
  - TensorBoard (for visualization)

- **Running the tuner**:  
  Trains models on the training data (`X_train`, `y_train`) with validation on (`X_valid`, `y_valid`), searching for the best hyperparameters.

- **Saving results**:  
  After tuning, the best hyperparameters are printed and saved to a pickle file.  
  The best model is saved in Keras format for later use.

---

This approach helps find the optimal neural network architecture and training settings for regression tasks automatically.


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import os
import pickle

def build_model_kt(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3, default=2)
    n_neurons = hp.Int("n_neurons", min_value=1, max_value=100, step=1)
    learning_rate = hp.Float("learning_rate", min_value=3e-4, max_value=3e-2, sampling="log")
    optimizer_name = hp.Choice("optimizer", values=["adam", "sgd", "nesterov"])

    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[8]))

    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))

    model.add(keras.layers.Dense(1))

    if optimizer_name == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_name == "nesterov":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])
    return model

# setting up catalouge for tensorbaord
project_name = "my_california_housing"
directory = "my_california_housing_dir"

# random search tuner initialization
random_search_tuner = kt.RandomSearch(
    build_model_kt,
    objective="val_mse",
    max_trials=20,
    executions_per_trial=1,
    directory=directory,
    project_name=project_name,
    overwrite=True,
    seed=42,
)

# Callback = EarlyStopping and TensorBoard
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
tensorboard_logdir = os.path.join(directory, project_name, "tensorboard_logs")
tensorboard_cb = keras.callbacks.TensorBoard(tensorboard_logdir)

# we start tuning parameters
random_search_tuner.search(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb, tensorboard_cb],
    verbose=2,
)

# we show the best parameters
best_hp = random_search_tuner.get_best_hyperparameters(num_trials=1)[0]
print("Najlepsze hiperparametry:")
print(f"n_hidden: {best_hp.get('n_hidden')}")
print(f"n_neurons: {best_hp.get('n_neurons')}")
print(f"learning_rate: {best_hp.get('learning_rate')}")
print(f"optimizer: {best_hp.get('optimizer')}")

# saving best parameters
best_hp_dict = {
    "n_hidden": best_hp.get("n_hidden"),
    "n_neurons": best_hp.get("n_neurons"),
    "learning_rate": best_hp.get("learning_rate"),
    "optimizer": best_hp.get("optimizer"),
}
with open("kt_search_params.pkl", "wb") as f:
    pickle.dump(best_hp_dict, f)

# build and save best model
best_model = random_search_tuner.get_best_models(num_models=1)[0]
best_model.save("kt_best_model.keras")